In [1]:
from hello import hello, hello3
from officelib.xllib import *
from officelib import const

In [17]:
HERR = (hello.HelloError, hello3.HelloError)
NLI = (hello.HelloError, hello3.NotLoggedInError)

from time import sleep
def set_mg(sp):
    call(app.setmg, 1, sp)        
        
def call(f, *args):
    
    while True:
        try:
            rv = f(*args)
        except NLI:
            app.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        else:
            return rv
        
        
def update_ss(cell, val):
    cell.Value = val
    c = cell.Offset(2, 1)
    c.Select()
    return c
        
def frange(start, stop, step):
    i = 0
    v = start + i * step
    while v < stop:
        yield v
        i += 1
        v = start + i * step
        

def run_test(name, mfc_max, c=None):
    tmin = mfc_max / 10
    print("Testing ", name, "MFC: %.1f LPM Max" % mfc_max)
    app.login()
    for fr in frange(tmin, mfc_max+tmin/2, tmin):
        set_mg(fr)
        v = input("%.2f LPM:" % fr)
        if v.lower() in ("break", "quit", "q", "stop"):
            break
        c = update_ss(c, v)

def air_on(v):
    app.setph(2,0,0)
    app.setdo(2,0,0)
    
def n2_on(v):
    app.setph(2,0,0)
    app.setdo(1, 100, 0)
    
def o2_on(v):
    app.setph(2,0,0)
    app.setdo(1, 0, 100)
    
def co2_on(v):
    app.setconfig("pH", "CO2_Manual_Max_(%)", 100)
    app.setph(1, 100, 0)
    app.setdo(2, 0, 0)
    app.setmg(1, v)
    input("Purging line of air. Press enter when done.")
    

cc = {
    "co2": 4,
    "air": 1,
    "n2": 2,
    "o2": 3
}

on_fn = {
    "co2": co2_on,
    "air": air_on,
    "n2": n2_on,
    "o2": o2_on
}

    
def s():
    return xl.Selection

def run(name, mfcm):
    c = cc[name]
    app.login()
    if name == "co2":
        co2m = app.getConfig()['pH']['CO2_Manual_Max_(%)']
    on_fn[name](mfcm)
    run_test(name, mfcm, cells(2,c))
    if name == "co2":
        app.login()
        app.setconfig("pH", "CO2_Manual_Max_(%)", co2m)

def all_off():
    app.login()
    app.setag(0, 12)
    app.setpumpa(0, 0)
    app.setpumpb(0, 0)
    app.setpumpc(0, 0)
    app.setpumpsample(0, 0)
    
def connect():
    global app
    v = input("Version: ").lower()
    ip = input("Enter IP Address: ")
    if v == "3":
        app = hello3.HelloApp(ip)
    else:
        app = hello.HelloApp(ip)
    app.getConfig = types.MethodType(getConfig, app)
    
def init():
    global xl, wb, ws, cells, cell_range
    xl, wb, ws, cells = xlObjs()
    cell_range = cells.Range
    cell_range("A1:D1").Value = [("Air", "N2", "O2", "CO2")]
    

In [7]:
from hello.hello3 import HelloXML
def getConfig(self):
    query = "?&call=getConfig"
    rsp = self.send_request(query)
    xml = HelloXML(rsp)
    self._verify_xml(xml)
    # James renamed the clusters to SV_Air
    # and SV_Mag to make it easier for him
    # to keep track of. The "proper" solution
    # would be to just return the proper XML 
    # element when parsing. The easiest solution
    # is to just check for the key, since conversion
    # from XML to dict occurs upstream. 
    try:
        return xml.data['System_Variables_Mag']
    except KeyError:
        return xml.data['System_Variables']
import types

In [13]:
init()
connect()
all_off()

Version: 3
Enter IP Address: 192.168.1.13


Error processing request: HTTPSConnectionPool(host='192.168.1.13', port=443): Max retries exceeded with url: /webservice/interface/?&call=login&val1=user1&val2=12345&json=1 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x000002DC11EAEB00>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))
Error processing request: HTTPSConnectionPool(host='192.168.1.13', port=443): Max retries exceeded with url: /webservice/interface/?&call=login&val1=user1&val2=12345&json=1 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x000002DC10C04198>: Failed to establish a new connection: [WinError 10053] An established connection was aborted by the software in your host machine',))


In [19]:
run("air", 2)

Testing  air MFC: 2.0 LPM Max
0.20 LPM:.233
0.40 LPM:.438
0.60 LPM:.643
0.80 LPM:.846
1.00 LPM:1.045
1.20 LPM:1.246
1.40 LPM:1.442
1.60 LPM:1.646
1.80 LPM:1.845
2.00 LPM:2.022


In [20]:
run("n2", 2)

Testing  n2 MFC: 2.0 LPM Max
0.20 LPM:0.234
0.40 LPM:.438
0.60 LPM:0.642
0.80 LPM:0.848
1.00 LPM:1.050
1.20 LPM:1.248
1.40 LPM:1.452
1.60 LPM:1.654
1.80 LPM:1.857
2.00 LPM:2.033


In [18]:
run("o2", .5)

Testing  o2 MFC: 0.5 LPM Max
0.05 LPM:59.1
0.10 LPM:110.0
0.15 LPM:161.0
0.20 LPM:212.2
0.25 LPM:263.8
0.30 LPM:316.4
0.35 LPM:367.0
0.40 LPM:417.0
0.45 LPM:466.6
0.50 LPM:509.6


In [21]:
run("co2", .3)

Purging line of air. Press enter when done.
Testing  co2 MFC: 0.3 LPM Max
0.03 LPM:34.6
0.06 LPM:65.8
0.09 LPM:96.9
0.12 LPM:127.6
0.15 LPM:159.3
0.18 LPM:190.8
0.21 LPM:222.0
0.24 LPM:253.8
0.27 LPM:285.2
0.30 LPM:313.2
